<a href="https://colab.research.google.com/github/kwojtylko/flat_price_estimator/blob/main/2_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Analiza danych

In [1]:
# import bibliotek
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np

In [58]:
# wczytywanie danych
gratka_df = pd.read_csv("gratka_dane_new.csv")

In [59]:
pd.set_option('max_colwidth', 400)
gratka_df

,url_base,url_page,page,location,title,link,description,area_m2,n_rooms,n_floor,price_per_m2
0,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=1,1,"Kraków, Krowodrza, małopolskie","Nowe mieszkanie Kraków Krowodrza, ul. Pod Fortem",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-krowodrza-ul-pod-fortem/ob/14399332,"Poznajcie bliżej naszą inwestycję – Krowodrza Fort \n\nNasza nowa inwestycja Krowodrza Fort to mieszkania z rynku pierwotnego, głównie dwu – lub trzy pokojowe. Du",44.93,2,7.0,10700
1,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=1,1,"Kraków, Podgórze, małopolskie","Nowe mieszkanie Kraków Podgórze, ul. Golikówka",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-podgorze-ul-golikowka/ob/19878365,Kukla Nieruchomości prezentujeMieszkanie 2- pokojowe o powierzchni 42 m2 zlokalizowane w domu wolnostojącym przy ulicy Golikówka w dzielnicy Rybitwy. Mieszkanie,42.00,2,1.0,7024
2,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=1,1,"Kraków, Krowodrza, małopolskie","Nowe mieszkanie Kraków Krowodrza, ul. Pod Fortem",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-krowodrza-ul-pod-fortem/ob/14403264,"Poznajcie bliżej naszą inwestycję – Krowodrza Fort \n\nNasza nowa inwestycja Krowodrza Fort to mieszkania z rynku pierwotnego, głównie dwu – lub trzy pokojowe. Du",59.15,3,1.0,9400
3,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=1,1,"Kraków, Krowodrza, małopolskie","Nowe mieszkanie Kraków Krowodrza, ul. Pod Fortem",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-krowodrza-ul-pod-fortem/ob/14398368,"Poznajcie bliżej naszą inwestycję – Krowodrza Fort \n\nNasza nowa inwestycja Krowodrza Fort to mieszkania z rynku pierwotnego, głównie dwu – lub trzy pokojowe. Du",50.56,2,7.0,10300
4,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=1,1,"Kraków, Krowodrza, małopolskie","Nowe mieszkanie Kraków Krowodrza, ul. Pod Fortem",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-krowodrza-ul-pod-fortem/ob/14398296,"Poznajcie bliżej naszą inwestycję – Krowodrza Fort \n\nNasza nowa inwestycja Krowodrza Fort to mieszkania z rynku pierwotnego, głównie dwu – lub trzy pokojowe. Du",76.35,4,6.0,9700
...,...,...,...,...,...,...,...,...,...,...,...
3476,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=110,110,"Kraków, Płaszów, małopolskie","Nowe mieszkanie Kraków Płaszów, ul. Myśliwska",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-plaszow-ul-mysliwska/ob/17790039,W budynkach Myśliwska Solar Garden w Krakowie-Płaszowie na piętrach oraz na poziomie garażu zaplanowane są komórki lokatorskie w cenie 3250 zł/m2. Metraże komór,42.22,2,0.0,9995
3477,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=110,110,"Kraków, Płaszów, małopolskie","Nowe mieszkanie Kraków Płaszów, ul. Myśliwska",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-plaszow-ul-mysliwska/ob/17790031,W budynkach Myśliwska Solar Garden w Krakowie-Płaszowie na piętrach oraz na poziomie garażu zaplanowane są komórki lokatorskie w cenie 3250 zł/m2. Metraże komór,67.07,3,1.0,8916
3478,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=110,110,"Kraków, Płaszów, małopolskie","Nowe mieszkanie Kraków Płaszów, ul. Myśliwska",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-plaszow-ul-mysliwska/ob/17790023,W budynkach Myśliwska Solar Garden w Krakowie-Płaszowie na piętrach oraz na poziomie garażu zaplanowane są komórki lokatorskie w cenie 3250 zł/m2. Metraże 

In [60]:
gratka_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3481 entries, 0 to 3480
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   url_base      3481 non-null   object 
 1   url_page      3481 non-null   object 
 2   page          3481 non-null   int64  
 3   location      3481 non-null   object 
 4   title         3481 non-null   object 
 5   link          3481 non-null   object 
 6   description   3481 non-null   object 
 7   area_m2       3481 non-null   float64
 8   n_rooms       3481 non-null   int64  
 9   n_floor       3481 non-null   float64
 10  price_per_m2  3481 non-null   int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 299.3+ KB


In [61]:
gratka_df.describe()

,page,area_m2,n_rooms,n_floor,price_per_m2
count,3481.000000,3481.000000,3481.000000,3481.000000,3481.000000
mean,55.598966,55.448914,2.559609,2.392129,10631.935363
std,31.648347,26.350922,1.027479,2.197552,2941.337028
min,1.000000,13.300000,1.000000,0.000000,4999.000000
25%,28.000000,39.000000,2.000000,1.000000,8893.000000
50%,56.000000,51.080000,3.000000,2.000000,9940.000000
75%,83.000000,66.110000,3.000000,4.000000,11955.000000
max,110.000000,230.000000,6.000000,14.000000,40000.000000


In [62]:
#sprawdzenie duplikatów
gratka_df[gratka_df.duplicated()]

,url_base,url_page,page,location,title,link,description,area_m2,n_rooms,n_floor,price_per_m2


In [63]:
gratka_df["area_m2"].value_counts()

58.00    18
59.09    17
13.44    17
32.00    13
84.17    13
         ..
92.03     1
71.49     1
88.36     1
58.15     1
97.59     1
Name: area_m2, Length: 1521, dtype: int64

In [64]:
gratka_df["n_rooms"].value_counts()

3    1245
2    1171
1     529
4     399
5     114
6      23
Name: n_rooms, dtype: int64

In [65]:
gratka_df["n_floor"].value_counts()

1.0     836
0.0     658
2.0     634
3.0     463
4.0     365
5.0     230
6.0     100
7.0      70
8.0      57
9.0      41
10.0     12
13.0      6
14.0      3
11.0      3
12.0      3
Name: n_floor, dtype: int64

In [66]:
gratka_df["price_per_m2"].value_counts()

9200     50
10200    50
9000     45
8400     41
9500     40
         ..
10724     1
10734     1
10746     1
12795     1
12288     1
Name: price_per_m2, Length: 1519, dtype: int64

In [67]:
# średnia cena m2 mieszkania wg. metrażu
gratka_df.groupby("area_m2").agg(dict(price_per_m2 = "mean")).sort_values(by = "price_per_m2", ascending = False)

,price_per_m2
area_m2,
170.00,29412.0
25.01,25977.5
106.00,25377.0
225.35,25000.0
226.14,25000.0
...,...
91.05,5437.0
136.05,5250.0
76.64,5193.0


In [68]:
# średnia cena m2 mieszkania wg. liczby pokoi
gratka_df.groupby("n_rooms").agg(dict(price_per_m2 = "mean")).sort_values(by = "price_per_m2", ascending = False)

,price_per_m2
n_rooms,
6,18120.347826
1,13625.718336
2,10603.261315
5,9996.657895
3,9670.033735
4,9498.155388


In [69]:
# średnia cena m2 mieszkania wg. nr piętra, na którym mieszkanie się znajduje
gratka_df.groupby("n_floor").agg(dict(price_per_m2 = "mean")).sort_values(by = "price_per_m2", ascending = False)

,price_per_m2
n_floor,
10.0,13734.500000
9.0,12335.609756
14.0,12206.666667
13.0,12068.333333
12.0,11670.000000
11.0,11413.333333
4.0,11296.402740
3.0,11173.732181
5.0,11026.082609


In [70]:
# analiza ceny wg. metrażu
import plotly.express as px

px.box(gratka_df, 
       x = "area_m2", 
       y = "price_per_m2",
       title = "Cena 1m2 mieszkania w zależności od metrażu całego mieszkania")

In [71]:
# analiza ceny wg. ilości pokojów
import plotly.express as px

px.box(gratka_df, 
       x = "n_rooms", 
       y = "price_per_m2",
       title = "Cena 1m2 mieszkania w zależności od ilości pokoi")

In [72]:
# analiza ceny wg. ilości pokojów
import plotly.express as px

px.box(gratka_df, 
       x = "n_floor", 
       y = "price_per_m2",
       title = "Cena 1m2 mieszkania w zależności od nr piętra, na której znajduje się nieruchomość")

In [73]:
gratka_df.query("price_per_m2 == 40000 ")

,url_base,url_page,page,location,title,link,description,area_m2,n_rooms,n_floor,price_per_m2
1262,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=41,41,"Kraków, Stare Miasto, małopolskie","Nowe mieszkanie Kraków Stare Miasto, ul. św. Sebastiana",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-stare-miasto-ul-sw-sebastiana/ob/21010453,"STARE MIASTO - ATRAKCYJNA INWESTYCJA W CENTRUM KRAKOWA! Prezentowana ofertach to kameralna inwestycja 10 mieszkań , zlokalizowanych w historycznej cz",27.50,1,3.0,40000
1264,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny,https://gratka.pl/nieruchomosci/mieszkania/krakow/pierwotny?page=41,41,"Kraków, Stare Miasto, małopolskie","Nowe mieszkanie Kraków Stare Miasto, ul. św. Sebastiana",https://gratka.pl/nieruchomosci/nowe-mieszkanie-krakow-stare-miasto-ul-sw-sebastiana/ob/21010451,"STARE MIASTO - ATRAKCYJNA INWESTYCJA W CENTRUM KRAKOWA! Prezentowana ofertach to kameralna inwestycja 10 mieszkań , zlokalizowanych w historycznej cz",25.01,1,1.0,40000


In [79]:
gratka_df["district"] = gratka_df["location"].str.replace(",","")
gratka_df["district"] = gratka_df["district"].str.replace("Kraków","")
gratka_df["district"] = gratka_df["district"].str.replace("małopolskie","")
gratka_df["district"] = gratka_df["district"].str.strip()

In [83]:
gratka_df["district"].value_counts()

                          563
Podgórze                  375
Mistrzejowice             301
Płaszów                   280
Prądnik Biały             276
Grzegórzki                247
Krowodrza                 212
Zabłocie                  156
Wola Duchacka             108
Os. Prądnik Biały          92
Dębniki                    92
Bieżanów                   83
Stare Miasto               83
Prądnik Czerwony           57
Bronowice Małe             49
Wola Justowska             44
Górka Narodowa             42
Nowa Huta                  41
Czyżyny Stare              29
Salwator                   29
Azory                      25
Czyżyny                    25
Os. Prądnik Czerwony       24
Zwierzyniec                22
Bronowice                  20
Os. Kliny Zacisze          20
Os. Ruczaj                 18
Olsza                      17
Podgórze Stare             14
Dąbie                      13
Os. Złocień                11
Sidzina                    11
Kazimierz                   9
Kleparz   